## SMART CONTRACTS USING PY-ALGORAND

In [1]:
import json
from algosdk import account, algod, mnemonic, transaction, encoding

In [2]:
#First example

# generate an account
private_key, address = account.generate_account()
print("Private key:", private_key)
print("Address:", address)

# check if the address is valid
if encoding.is_valid_address(address):
    print("The address is valid!")
else:
    print("The address is invalid.")

Private key: UtmuX36KLtm5IRrqEJRLoMGqWuur5mSywQkbPv+H0bi5P4ijzX/tBpgrvcYtRAXepANB6iMGhjqoyJNFYbJAqQ==
Address: XE7YRI6NP7WQNGBLXXDC2RAF32SAGQPKEMDIMOVIZCJUKYNSICU7MW5UOQ
The address is valid!


In [3]:
#Create frase seed
mnemonic1 = "canal enact luggage spring similar zoo couple stomach shoe laptop middle wonder eager monitor weather number heavy skirt siren purity spell maze warfare ability ten"
mnemonic2 = "beauty nurse season autumn curve slice cry strategy frozen spy panic hobby strong goose employ review love fee pride enlist friend enroll clip ability runway"
mnemonic3 = "picnic bright know ticket purity pluck stumble destroy ugly tuna luggage quote frame loan wealth edge carpet drift cinnamon resemble shrimp grain dynamic absorb edge"


# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    accounts[counter]['pk'] = mnemonic.to_public_key(m)
    accounts[counter]['sk'] = mnemonic.to_private_key(m)
    counter += 1

In [4]:
# Specify your node address and token. This must be updated.
# algod_address = ""  # ADD ADDRESS
# algod_token = ""  # ADD TOKEN

algod_address = "http://hackathon.algodev.network:9100"
algod_token = "ef920e2e7e002953f4b29a8af720efe8e4ecc75ff102b165e0472834b25832c1"


In [5]:
# Initialize an algod client
algod_client = algod.AlgodClient(algod_token, algod_address)

In [6]:
# Get network params for transactions.
params = algod_client.suggested_params()
first = params.get("lastRound")
last = first + 1000
gen = params.get("genesisID")
gh = params.get("genesishashb64")
min_fee = params.get("minFee")

In [7]:
# Utility function to wait for a transaction to be confirmed by network

def wait_for_tx_confirmation(txid):
   last_round = algod_client.status().get('lastRound')
   while True:
       txinfo = algod_client.pending_transaction_info(txid)
       if txinfo.get('round') and txinfo.get('round') > 0:
           print("Transaction {} confirmed in round {}.".format(
               txid, txinfo.get('round')))
           break
       else:
           print("Waiting for confirmation...")
           last_round += 1
           algod_client.status_after_block(last_round)

In [8]:
print("Account 1 address: {}".format(accounts[1]['pk']))
print("Account 2 address: {}".format(accounts[2]['pk']))
print("Account 3 address: {}".format(accounts[3]['pk']))


Account 1 address: ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ
Account 2 address: AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4
Account 3 address: IWR4CLLCN2TIVX2QPVVKVR5ER5OZGMWAV5QB2UIPYMPKBPLJZX4C37C4AA


In [9]:
# Account 1 creates an asset called latinum and sets Account 2 as the manager, reserve, freeze, and clawback address.

data = {
    "sender": accounts[1]['pk'],
    "fee": min_fee,
    "first": first,
    "last": last,
    "gh": gh,
    "total": 1000,
    "default_frozen": False,
    "unit_name": "LATIMUN",
    "asset_name": "latimun",
    "manager": accounts[2]['pk'],
    "reserve": accounts[2]['pk'],
    "freeze": accounts[2]['pk'],
    "clawback": accounts[2]['pk'],
    "url": "https://path/to/my/asset/details",
    "flat_fee": True,
    "decimals": 0
}

In [10]:
# Construct Asset Creation transaction
txn = transaction.AssetConfigTxn(**data)

In [11]:
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])

In [12]:
# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)

NNVBNI7KJXKZETWWZ37ZBPYDLJBMABVRGJRWZSC2Y5PKJGNJMFUQ


In [13]:
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)

try:
    # Pull account info for the creator
    account_info = algod_client.account_info(accounts[1]['pk'])
    # Get max asset ID
    asset_id = max(
        map(lambda x: int(x), account_info.get('thisassettotal').keys()))
    print("Asset ID: {}".format(asset_id))
    print(json.dumps(account_info['thisassettotal'][str(asset_id)], indent=4))
except Exception as e:
    print(e)

Waiting for confirmation...
Transaction NNVBNI7KJXKZETWWZ37ZBPYDLJBMABVRGJRWZSC2Y5PKJGNJMFUQ confirmed in round 15109880.
Asset ID: 18050418
{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "total": 1000,
    "decimals": 0,
    "defaultfrozen": false,
    "unitname": "LATIMUN",
    "assetname": "latimun",
    "url": "https://path/to/my/asset/details",
    "managerkey": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "reserveaddr": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "freezeaddr": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "clawbackaddr": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4"
}


In [14]:
# asset_id = 328952;
data = {
    "sender": accounts[2]['pk'],
    "fee": min_fee,
    "first": first,
    "last": last,
    "gh": gh,
    "index": asset_id,
    "manager": accounts[1]['pk'],
    "reserve": accounts[2]['pk'],
    "freeze": accounts[2]['pk'],
    "clawback": accounts[2]['pk'],
    "flat_fee": True
}
txn = transaction.AssetConfigTxn(**data)
# sign by the current manager
stxn = txn.sign(accounts[2]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)

T5IXKOO4QXCF3V7SILCCDQ3HJE7K7IJPPZGUR57XMRTDHBZKNCWA


In [15]:
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)

Waiting for confirmation...
Transaction T5IXKOO4QXCF3V7SILCCDQ3HJE7K7IJPPZGUR57XMRTDHBZKNCWA confirmed in round 15109882.


In [16]:
# Check asset info to view change in management.
asset_info = algod_client.asset_info(asset_id)
print(json.dumps(asset_info, indent=4))

{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "total": 1000,
    "decimals": 0,
    "defaultfrozen": false,
    "unitname": "LATIMUN",
    "assetname": "latimun",
    "url": "https://path/to/my/asset/details",
    "managerkey": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "reserveaddr": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "freezeaddr": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "clawbackaddr": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4"
}


In [17]:
# opt-in
# Check if asset_id is in account 3's asset holdings prior to opt-in
account_info = algod_client.account_info(accounts[3]['pk'])
holding = None
if 'assets' in account_info:
    holding = account_info['assets'].get(str(asset_id))

if not holding:
    # Get latest network parameters
    data = {
        "sender": accounts[3]['pk'],
        "fee": min_fee,
        "first": first,
        "last": last,
        "gh": gh,
        "receiver": accounts[3]["pk"],
        "amt": 0,
        "index": asset_id,
        "flat_fee": True
    }

In [18]:
# Use the AssetTransferTxn class to transfer assets and opt-in
txn = transaction.AssetTransferTxn(**data)
stxn = txn.sign(accounts[3]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)
# Now check the asset holding for that account.
# This should now show a holding with a balance of 0.
account_info = algod_client.account_info(accounts[3]['pk'])
print(json.dumps(account_info['assets'][str(asset_id)], indent=4))

VU6BNHCOZAUUAHBS72DFT4TFHZTFVEMITJ46LGH5CYZHPYD564BA
Waiting for confirmation...
Transaction VU6BNHCOZAUUAHBS72DFT4TFHZTFVEMITJ46LGH5CYZHPYD564BA confirmed in round 15109884.
{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "amount": 0,
    "frozen": false
}


In [19]:
# transfer asset of 10 from account 1 to account 3
data = {
    "sender": accounts[1]['pk'],
    "fee": min_fee,
    "first": first,
    "last": last,
    "gh": gh,
    "receiver": accounts[3]["pk"],
    "amt": 10,
    "index": asset_id,
    "flat_fee": True
}

txn = transaction.AssetTransferTxn(**data)
stxn = txn.sign(accounts[1]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)

5R4BYLBWLOZCVW3FPYSPBSYHDWHZHGGZGA2SRRXHYU7MAMLQ6SDQ


In [20]:
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)
# The balance should now be 10.
account_info = algod_client.account_info(accounts[3]['pk'])
print(json.dumps(account_info['assets'][str(asset_id)], indent=4))

Waiting for confirmation...
Transaction 5R4BYLBWLOZCVW3FPYSPBSYHDWHZHGGZGA2SRRXHYU7MAMLQ6SDQ confirmed in round 15109886.
{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "amount": 10,
    "frozen": false
}


In [21]:
# The freeze address (Account 2) freezes Account 3's latinum holdings.
data = {
    "sender": accounts[2]['pk'],
    "fee": min_fee,
    "first": first,
    "last": last,
    "gh": gh,
    "index": asset_id,
    "target": accounts[3]["pk"],
    "new_freeze_state": True
}

txn = transaction.AssetFreezeTxn(**data)
stxn = txn.sign(accounts[2]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)
# The balance should now be 10.
account_info = algod_client.account_info(accounts[3]['pk'])
print(json.dumps(account_info['assets'][str(asset_id)], indent=4))

DTWHWE3JH7VMHRBPT3G5PLSWHQ7VZHA5ELPGZTI6AUG65VPPH7WA
Waiting for confirmation...
Transaction DTWHWE3JH7VMHRBPT3G5PLSWHQ7VZHA5ELPGZTI6AUG65VPPH7WA confirmed in round 15109888.
{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "amount": 10,
    "frozen": true
}


In [22]:
# The clawback address (Account 2) revokes 10 latinum from Account 3 and places it back with Account 1.
data = {
    "sender": accounts[2]['pk'],
    "fee": min_fee,
    "first": first,
    "last": last,
    "gh": gh,
    "receiver": accounts[1]["pk"],
    "amt": 10,
    "index": asset_id,
    "revocation_target": accounts[3]['pk'],
    "flat_fee": True
}
# Must be signed by the account that is the clawback address
txn = transaction.AssetTransferTxn(**data)
stxn = txn.sign(accounts[2]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)
# The balance of account 3 should now be 0.
account_info = algod_client.account_info(accounts[3]['pk'])
print("Account 3")
print(json.dumps(account_info['assets'][str(asset_id)], indent=4))
# The balance of account 1 should increase by 10 to 1000.
print("Account 1")
account_info = algod_client.account_info(accounts[1]['pk'])
print(json.dumps(account_info['assets'][str(asset_id)], indent=4))

DTICZVPTUPXM6A4AGVDGVBUF5XR4BDJGIANDWXDQS5SECSMSFLXQ
Waiting for confirmation...
Transaction DTICZVPTUPXM6A4AGVDGVBUF5XR4BDJGIANDWXDQS5SECSMSFLXQ confirmed in round 15109890.
Account 3
{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "amount": 0,
    "frozen": true
}
Account 1
{
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "amount": 1000,
    "frozen": false
}


In [23]:
# With all assets back in the creator's account,
# the manager (Account 1) destroys the asset.

data = {
    "sender": accounts[1]['pk'],
    "fee": min_fee,
    "first": first,
    "last": last,
    "gh": gh,
    "index": asset_id,
    "strict_empty_address_check": False,
    "flat_fee": True
}


In [24]:
# Construct Asset Creation transaction
txn = transaction.AssetConfigTxn(**data)

In [25]:
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])


In [26]:
# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)

GR5VTHKY7TXBSER4VLTAGVLJG6MDWXUYQQGDAH6F54R42SWKEQIQ


In [27]:
# Wait for the transaction to be confirmed
wait_for_tx_confirmation(txid)

Waiting for confirmation...
Transaction GR5VTHKY7TXBSER4VLTAGVLJG6MDWXUYQQGDAH6F54R42SWKEQIQ confirmed in round 15109892.


In [28]:
# This should raise an exception since the asset was deleted.
try:
    asset_info = algod_client.asset_info(asset_id)
except Exception as e:
    print(e)

failed to retrieve asset creator from the ledger


In [29]:
acct = account.generate_account()
address1 = acct[1]
print("Account 1")
print(address1)
mnemonic1 = mnemonic.from_private_key(acct[0])

print("Account 2")
acct = account.generate_account()
address2 = acct[1]
print(address2)
mnemonic2 = mnemonic.from_private_key(acct[0])

print("Account 3")
acct = account.generate_account()
address3 = acct[1]
print(address3)
mnemonic3 = mnemonic.from_private_key(acct[0])
print ("")
print("Copy off accounts above and add TestNet Algo funds using the TestNet Dispenser at https://bank.testnet.algorand.network/")
print("copy off the following mnemonic code for use later")
print("")
print("mnemonic1 = \"{}\"".format(mnemonic1))
print("mnemonic2 = \"{}\"".format(mnemonic2))
print("mnemonic3 = \"{}\"".format(mnemonic3))

Account 1
FTLUAPKIILXOITRSJHILW3I7U2PHANIRVEHWZ2SXT3Y6MNJBDOJJVPFBOE
Account 2
UGR33FU65BX5ZHIN2OROILJ3CBCKTSTMDPZBY322BZ33NOSRS72PR5LZNM
Account 3
V3MFRTI7FTRCSZSVJNTNUAHZIIR5V7VFZ6VBAILTPFDEQ2RZYQW2UO22HU

Copy off accounts above and add TestNet Algo funds using the TestNet Dispenser at https://bank.testnet.algorand.network/
copy off the following mnemonic code for use later

mnemonic1 = "spray lunch scorpion ritual nature spread domain hen biology asset audit congress mistake banner ready robot glory dial identify coast gift clarify verb absorb gloom"
mnemonic2 = "lift final supply blouse essence glow follow spike unveil critic next helmet case certain balcony useless couple cup clump betray flat glance cancel able divorce"
mnemonic3 = "design melody earth cabbage exclude emerge helmet alone buddy laugh rug able reopen blame mask kick staff version expect decide urge high brand abandon toddler"
